<a href="https://colab.research.google.com/github/nicanwjdiwiad/Vison-Transformer/blob/main/ViTipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#第一步：把图片变为embedding

#DNN版本实现

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f

#将图片转为embedding
#weight是把token(patch)线性变换转为embedding的权重 patch*W=embedding
def image2emb_naive(image, patch_size, weight):
  #image_size = batchsize*channel*h*w 像素点个数
  #使用拿出卷积区域的api，把每次滑动的小区域的输入拿出来
  #因为图像没有重叠的部分，所以说滑动卷积核步长stride=块边长
  patch = f.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1,-2)
  # print(patch.shape)
  #patch.Size([1, 4, 48])即有1个4*48大小的矩阵，48个像素表示一个patch，4个patch表示一张图片
  #reason: 1:image数量
  #    4:一个图片中patch数量
  #    48:一个patch中像素数量=patch面积*channel

  patch_embedding = patch @ weight
  return patch_embedding


#test code 4 image2emb(patch的size)
batch_size,inchannel,image_h,image_w=1,3,8,8

patch_size=4

model_dim=8

patch_xiangsu_number = patch_size*patch_size*inchannel

image = torch.randn(batch_size,inchannel,image_h,image_w)

weight = torch.randn(patch_xiangsu_number,model_dim)

# print(weight.shape)  #weight.Size([48, 8])

patch_embedding_naive = image2emb_naive(image, patch_size, weight)

print(patch_embedding_naive.shape) #torch.Size([1, 4, 8]),可以明显看出一个patch一个embedding,且向量长度为d_model

torch.Size([1, 4, 8])


#CNN版本实现

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as f

def image2emb_conv(image, kernel, stride):
  #1.卷积，得到输出尺寸batchsize*channel*h*w
  conv_output=f.conv2d(image, kernel, stride=stride)
  bs,oc,oh,ow=conv_output.shape
  #2.把卷积结果拉直
  patch_embedding=conv_output.reshape((bs,oc,oh*ow)).transpose(-1,-2)
  return patch_embedding

#test code 4 image2emb(patch的size)
batch_size,inchannel,image_h,image_w=1,3,8,8

patch_size=4

model_dim=8

max_num_token = 16 #序列最长时token数,位置编码行数上界

num_classes = 10 #最后目标分类数量

label = torch.randn(batch_size,10) #随机生成10个数模拟真实类标签

patch_xiangsu_number = patch_size*patch_size*inchannel

image = torch.randn(batch_size,inchannel,image_h,image_w)

weight = torch.randn(patch_xiangsu_number,model_dim) #这里把模型维度视为out_channel.patch_xiangsu_number是卷积核面积*IN_channel

# print(weight.shape)  #weight.Size([48, 8])

kernel=weight.transpose(0,1).reshape((model_dim,inchannel,patch_size,patch_size)) #卷积核维度oc*ic*kh*kw
patch_embedding_conv = image2emb_conv(image, kernel, patch_size)
print(patch_embedding_conv.shape)

torch.Size([1, 4, 8])


#第二步：增加一个Query(cls)去指导embedding的表示(最后是分类任务)
CLS token embedding

In [9]:
cls_token_embedding = torch.randn(batch_size,1,model_dim,requires_grad=True) #有梯度表示可训练的embedding
print(cls_token_embedding.shape) #torch.Size([1, 1, 8])
#将指导emb与原emb concat
visual_token_embedding = torch.cat([cls_token_embedding,patch_embedding_conv],dim=1)#dim=1表示在位置上去cat
print(visual_token_embedding.shape) #torch.Size([1, 5, 8])
#这里可以理解为cls视为图像的分类特征，此时图像有5个patch，所以加载dim=1维度

torch.Size([1, 1, 8])
torch.Size([1, 5, 8])


#第三步加位置编码

In [11]:
position_embedding_table = torch.randn(max_num_token,model_dim,requires_grad=True)
seq_len = visual_token_embedding.shape[1] #这里是5,表示5个patch需要位置编码
position_embedding=torch.tile(position_embedding_table[:seq_len],[visual_token_embedding.shape[0],1,1]) #表示dim=0是visual_token_embedding.shape[0]=1，dim=1==position_embedding_table[:seq_len][0],dim=2==position_embedding_table[:seq_len][1]
visual_token_embedding += position_embedding

#第四步：将embedding送入transformer encoder

In [13]:
encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim,nhead=8) #8个attention头
transformer_encoder=nn.TransformerEncoder(encoder_layer,num_layers=6)  #6层encoder
encoder_output = transformer_encoder(visual_token_embedding)
print(encoder_output.shape)

torch.Size([1, 5, 8])


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


#与cls做交叉熵，做分类

In [32]:
cls_token_output = encoder_output[:,0,:]
print(cls_token_output.shape) #torch.Size([1, 8]) 将patch个数维度删掉，仅考虑模型维度和批量大小
linear_layer = nn.Linear(model_dim,num_classes)
logits = linear_layer(cls_token_output)
loss_fn = nn.CrossEntropyLoss()  #封装了softmax和交叉熵损失函数
loss = loss_fn(logits_probs,label)
print(loss)

torch.Size([1, 8])
tensor(-11.7735, grad_fn=<DivBackward1>)
